<a href="https://colab.research.google.com/github/KunjShah95/DEEPFAKE-DETECTION-SYSTEM-/blob/main/UT1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install pandas numpy matplotlib scikit-learn
%pip install tensorflow opencv-python matplotlib transformers

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

import os
import cv2
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import kagglehub

# Download latest version
path = kagglehub.dataset_download("manjilkarki/deepfake-and-real-images")

print("Path to dataset files:",path)




100%|██████████| 1.68G/1.68G [00:18<00:00, 98.9MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/manjilkarki/deepfake-and-real-images/versions/1


In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG16  # or any other base model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# Load a pre-trained base model (e.g., VGG16)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add a fully connected layer
x = Dense(1024, activation='relu')(x)

# Add an output layer (e.g., for binary classification)
output_layer = Dense(1, activation='sigmoid')(x)  # Change the number of units and activation as needed

# Define the model
model = Model(inputs=base_model.input, outputs=output_layer)

# Optionally, compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Now the model is ready to be used

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [4]:
import tensorflow as tf
# Unfreeze last few layers for fine-tuning
for layer in base_model.layers[-20:]:
    layer.trainable = True

# Recompile after unfreezing
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),
              loss="binary_crossentropy",
              metrics=["accuracy"])


In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data augmentation
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=30, zoom_range=0.2,
                                   horizontal_flip=True, validation_split=0.2)

train_generator = train_datagen.flow_from_directory("/root/.cache/kagglehub/datasets/manjilkarki/deepfake-and-real-images/versions/1",
                                                    target_size=(224, 224),
                                                    batch_size=32,
                                                    class_mode="binary",
                                                    subset="training")

val_generator = train_datagen.flow_from_directory("/root/.cache/kagglehub/datasets/manjilkarki/deepfake-and-real-images/versions/1",
                                                  target_size=(224, 224),
                                                  batch_size=32,
                                                  class_mode="binary",
                                                  subset="validation")



Found 152268 images belonging to 1 classes.
Found 38067 images belonging to 1 classes.


In [6]:
# Train the model
model.fit(train_generator, validation_data=val_generator, epochs=10)

# Ensure evaluation input size matches model input size
val_generator = data_gen.flow_from_directory("/root/.cache/kagglehub/datasets/manjilkarki/deepfake-and-real-images/versions/1",  # Use the correct dataset path variable
                                             target_size=(224, 224),
                                             batch_size=32,
                                             class_mode='binary',
                                             subset='validation',
                                             shuffle=False)



/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
   4/4759 ━━━━━━━━━━━━━━━━━━━━ 96:35:12 73s/step - accuracy: 1.0000 - loss: 0.4404

KeyboardInterrupt: 

In [ ]:
test_loss, test_acc = model.evaluate(val_generator)
print(f"Test Accuracy: {test_acc:.2f}")